<a href="https://colab.research.google.com/github/Pitou11/fairness2/blob/main/UPP26/TD4_pre_post.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TD 4: Mitigation des biais avec des méthodes de pré-processing et de post-processing



## Installation of the environnement

We highly recommend you to follow these steps, it will allow every student to work in an environment as similar as possible to the one used during testing.

### Colab Settings ---- for Colab Users ONLY
  The next cell of code are to execute only once per colab environment


#### Python env creation (Colab only)

        ```
        ! python -m pip install numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit
        ```
#### 2. Download MEPS dataset (for part2) it can take several minutes (Colab only)

        ```
        ! Rscript /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/generate_data.R
        ! mv h181.csv /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/
        ! mv h192.csv /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/
        ```

### Local Settings ---- for installation on local computer ONLY

If you arleady have an env from TD2 or TD3, you can simply reuse it.


#### 1. Uv installation (local only, no need to redo if already done)


        https://docs.astral.sh/uv/getting-started/installation/


        `curl -LsSf https://astral.sh/uv/install.sh | sh`

        Python version 3.12 installation (highly recommended)
        `uv python install 3.12`

#### 2. R installation *NEW* (local only)

        In the command `Rscript` says 'command not found'

        `sudo apt install r-base-core`

#### 3. Python env creation (local only, no need to redo if already done)

        ```
        mkdir TD_bias_mitigation
        cd TD_bias_mitigation
        uv python pin 3.12
        uv init
        uv venv
        uv add numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit
        uv add pandas==2.2.2
        ```

#### 4. Download MEPS dataset, it can take several minutes *NEW* (local only)

        ```
        cd TD_bias_mitigation/.venv/lib/python3.12/site-packages/aif360/data/raw/meps/
        Rscript generate_data.R
        ```

In [1]:
# To execute only in Colab
! python -m pip install numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.8/191.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# To execute only in Colab
! Rscript /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/generate_data.R
! mv h181.csv /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/
! mv h192.csv /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/


By using this script you acknowledge the responsibility for reading and
abiding by any copyright/usage rules and restrictions as stated on the
MEPS web site (https://meps.ahrq.gov/data_stats/data_use.jsp).

Continue [y/n]? > y
Loading required package: foreign
trying URL 'https://meps.ahrq.gov/mepsweb/data_files/pufs/h181ssp.zip'
Content type 'application/zip' length 13303652 bytes (12.7 MB)
downloaded 12.7 MB

Loading dataframe from file: h181.ssp
Exporting dataframe to file: h181.csv
trying URL 'https://meps.ahrq.gov/mepsweb/data_files/pufs/h192ssp.zip'
Content type 'application/zip' length 15505898 bytes (14.8 MB)
downloaded 14.8 MB

Loading dataframe from file: h192.ssp
Exporting dataframe to file: h192.csv


## 1.Manipulate the dataset

In [3]:
# imports
import numpy as np
import pandas as pd
import plotly.express as px
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", append=True, category=UserWarning)
# Datasets
from aif360.datasets import MEPSDataset19
from aif360.explainers import MetricTextExplainer

# Fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from sklearn.metrics import accuracy_score, balanced_accuracy_score


MEPSDataset19_data = MEPSDataset19()
(dataset_orig_panel19_train, dataset_orig_panel19_val, dataset_orig_panel19_test) = (
    MEPSDataset19().split([0.5, 0.8], shuffle=True)
)

In [4]:
len(dataset_orig_panel19_train.instance_weights), len(
    dataset_orig_panel19_val.instance_weights
), len(dataset_orig_panel19_test.instance_weights)

(7915, 4749, 3166)

In [5]:
instance_weights = MEPSDataset19_data.instance_weights
instance_weights

array([21854.981705, 18169.604822, 17191.832515, ...,  3896.116219,
        4883.851005,  6630.588948])

In [6]:
f"Taille du dataset {len(instance_weights)}, poids total du dataset {instance_weights.sum()}."

'Taille du dataset 15830, poids total du dataset 141367240.546316.'

Conversion en dataframe

In [7]:
def get_df(MepsDataset):
    data = MepsDataset.convert_to_dataframe()
    # data_train est un tuple, avec le data_frame et un dictionnaire avec toutes les infos (poids, attributs sensibles etc)
    df = data[0]
    df["WEIGHT"] = data[1]["instance_weights"]
    return df


df = get_df(MEPSDataset19_data)

Nous réalisons maintenant l'opération inverse (qui sera indispensable pour le projet). Créer un objet de la classe StandardDataset de AIF360 à partir du dataframe.

Pour le projet cela vous permettre d'utiliser les méthode déjà implémentées dans AIF360 sur votre jeu de données.

Ici cela n'a aucun intéret car le dataframe vien d'un StandardDataset, nous vous fournissons le code. Mais cela vaut le coup de le lire attentivement et de poser des questions si besoin.




In [8]:
import os
from aif360.datasets import StandardDataset
import pandas as pd

# Get categorical column from one hot encoding (specitic to MEPSdataset)
# Here we create a dictionnary that links each categorical column name
# to the list of corresponding one hot encoded columns
categorical_columns_dic = {}
for col in df.columns:
    col_split = col.split("=")
    if len(col_split) > 1:
        cat_col = col_split[0]
        if not (cat_col in categorical_columns_dic.keys()):
            categorical_columns_dic[cat_col] = []
        categorical_columns_dic[cat_col].append(col)
categorical_features = categorical_columns_dic.keys()

In [9]:
# Now we recreate the categorical column value from the one hot encoded
print(df.shape)


def categorical_transform(df, onehotencoded, cat_col):
    if len(onehotencoded) > 1:
        return df[onehotencoded].apply(
            lambda x: onehotencoded[np.argmax(x)][len(cat_col) + 1 :], axis=1
        )
    else:
        return df[onehotencoded]


# Reverse the categorical one hot encoded
for cat_col, onehotencoded in categorical_columns_dic.items():
    df[cat_col] = categorical_transform(df, onehotencoded, cat_col)
    df.drop(columns=onehotencoded, inplace=True)

df.shape

(15830, 140)


(15830, 43)

In [10]:
MyDataset = StandardDataset(
    df=df,
    label_name="UTILIZATION",
    favorable_classes=[1],
    protected_attribute_names=["RACE"],
    privileged_classes=[[1]],
    instance_weights_name="WEIGHT",
    categorical_features=categorical_features,
    features_to_keep=[],
    features_to_drop=[],
    na_values=["?", "Unknown/Invalid"],
    custom_preprocessing=None,
    metadata=None,
)

In [11]:
# We check the dataset has the same metrics :D
# Attention étonnanement le positive label 'favorable_classes' est par défaut 1 (cela est un peu bizarre pour ce dataset)
print(
    BinaryLabelDatasetMetric(
        MEPSDataset19_data,
        unprivileged_groups=[{"RACE": 0}],
        privileged_groups=[{"RACE": 1}],
    ).disparate_impact(),
    BinaryLabelDatasetMetric(
        MEPSDataset19_data,
        unprivileged_groups=[{"RACE": 0}],
        privileged_groups=[{"RACE": 1}],
    ).base_rate(),
)
print(
    BinaryLabelDatasetMetric(
        MyDataset, unprivileged_groups=[{"RACE": 0}], privileged_groups=[{"RACE": 1}]
    ).disparate_impact(),
    BinaryLabelDatasetMetric(
        MyDataset, unprivileged_groups=[{"RACE": 0}], privileged_groups=[{"RACE": 1}]
    ).base_rate(),
)

0.49826823461176517 0.21507139363038463
0.49826823461176517 0.21507139363038463


In [12]:
from aif360.sklearn.metrics import disparate_impact_ratio, base_rate

dir = disparate_impact_ratio(
    y_true=df.UTILIZATION, prot_attr=df.RACE, pos_label=1, sample_weight=df.WEIGHT
)
br = base_rate(y_true=df.UTILIZATION, pos_label=1, sample_weight=df.WEIGHT)
dir, br

(0.4982682346117653, np.float64(0.21507139363038463))

## 2. Appliquer les méthodes de pré-processing disponibles dans AIF360

In [13]:
sens_ind = 0
sens_attr = dataset_orig_panel19_train.protected_attribute_names[sens_ind]
unprivileged_groups = [
    {sens_attr: v}
    for v in dataset_orig_panel19_train.unprivileged_protected_attributes[sens_ind]
]
privileged_groups = [
    {sens_attr: v}
    for v in dataset_orig_panel19_train.privileged_protected_attributes[sens_ind]
]
sens_attr, unprivileged_groups, privileged_groups

('RACE', [{'RACE': np.float64(0.0)}], [{'RACE': np.float64(1.0)}])

### 2.1 Quesiton: Apprendre une regression logistique qui prédit l'UTILIZATION

Attention nous avons enlever le preprocessing sur le dataframe, il faut cette fois utiliser l'API d'AIF360
https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.StructuredDataset.html

pour retrouver les features (X), les labels (y) et les poids de chaque instance du dataset

In [29]:
#training set
X_train = dataset_orig_panel19_train.features
y_train = dataset_orig_panel19_train.labels.ravel()
w_train = dataset_orig_panel19_train.instance_weights.ravel()

#validation set
X_val = dataset_orig_panel19_val.features
y_val = dataset_orig_panel19_val.labels.ravel()
w_val = dataset_orig_panel19_val.instance_weights.ravel()


print(X_train.shape, y_train.shape, w_train.shape)
print(X_val.shape, y_val.shape, w_val.shape)

(7915, 138) (7915,) (7915,)
(4749, 138) (4749,) (4749,)


### 2.2 Question: Calcul des métriques de fairness

Calculer les métriques du dataset de validation seul.

Calculer les métriques basées sur les prédictions et la vérité du dataset de validation.

En comparaison calculer les métriques basées sur des prédictions aléatoires et la vérité du dataset de validation.

In [27]:
print("Metrics on validation dataset")

metric_val = BinaryLabelDatasetMetric(
    dataset_orig_panel19_val,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

print(f"  Disparate Impact: {metric_val.disparate_impact():.4f}")
print(f"  Base Rate (privileged): {metric_val.base_rate(privileged=True):.4f}")
print(f"  Base Rate (unprivileged): {metric_val.base_rate(privileged=False):.4f}")
print(f"  Base Rate (overall): {metric_val.base_rate():.4f}")

Metrics on validation dataset
  Disparate Impact: 0.4677
  Base Rate (privileged): 0.2877
  Base Rate (unprivileged): 0.1346
  Base Rate (overall): 0.2279


In [33]:
print("Metrics based on predictions and truth on validation dataset")
metric_basedonpred_val = ClassificationMetric( )


print(f"  Accuracy: {metric_basedonpred_val.accuracy():.4f}")
print(f"  Balanced Accuracy: {metric_basedonpred_val.balanced_accuracy():.4f}")
print(f"  Disparate Impact: {metric_basedonpred_val.disparate_impact():.4f}")

Metrics based on predictions and truth on validation dataset


NameError: name 'dataset_orig_panel19' is not defined

In [35]:
from sklearn.linear_model import LogisticRegression

# Train a Logistic Regression model
# We use solver='liblinear' for better compatibility with small datasets or weighted samples
model = LogisticRegression(solver='liblinear', random_state=42)
model.fit(X_train, y_train, sample_weight=w_train)

# Make predictions on the validation set
y_pred_val = model.predict(X_val)
y_prob_val = model.predict_proba(X_val)[:, 1]

print("Logistic Regression model trained and predictions made on validation set.")

Logistic Regression model trained and predictions made on validation set.


Maintenant que nous avons les prédictions, nous pouvons créer le `dataset_pred_val` nécessaire à `ClassificationMetric`.

In [37]:
from aif360.datasets import BinaryLabelDataset

# Create a BinaryLabelDataset for predictions on the validation set
dataset_pred_val = dataset_orig_panel19_val.copy(deepcopy=True)
dataset_pred_val.labels = y_pred_val.reshape(-1, 1)

# Now you can use ClassificationMetric
print("Metrics based on predictions and truth on validation dataset")
metric_basedonpred_val = ClassificationMetric(
    dataset_orig_panel19_val, # Ground truth dataset
    dataset_pred_val,       # Predicted labels dataset
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

print(f"  Accuracy: {metric_basedonpred_val.accuracy():.4f}")
print(f"  Base Rate: {metric_basedonpred_val.base_rate():.4f}")
print(f"  Disparate Impact: {metric_basedonpred_val.disparate_impact():.4f}")

Metrics based on predictions and truth on validation dataset
  Accuracy: 0.8344
  Base Rate: 0.2279
  Disparate Impact: 0.3703


In [38]:
import numpy as np
from aif360.datasets import BinaryLabelDataset

# Generate random predictions for the validation set
# We'll make sure the shape matches y_val
y_pred_random = np.random.randint(0, 2, size=y_val.shape).reshape(-1, 1)

# Create a BinaryLabelDataset for random predictions on the validation set
dataset_pred_random = dataset_orig_panel19_val.copy(deepcopy=True)
dataset_pred_random.labels = y_pred_random

print("Metrics based on random predictions and truth on validation dataset")
metric_random_pred_val = ClassificationMetric(
    dataset_orig_panel19_val, # Ground truth dataset
    dataset_pred_random,      # Randomly predicted labels dataset
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

print(f"  Accuracy (Random): {metric_random_pred_val.accuracy():.4f}")
print(f"  Base Rate (Random): {metric_random_pred_val.base_rate():.4f}")
print(f"  Disparate Impact (Random): {metric_random_pred_val.disparate_impact():.4f}")

Metrics based on random predictions and truth on validation dataset
  Accuracy (Random): 0.5050
  Base Rate (Random): 0.2279
  Disparate Impact (Random): 1.0752


### 2.2 Repondération
#### 2.2.1. Question : Trouver dans l'API quels objets/fonctions sont à utiliser pour faire de repondération et les appliquer sur le dataset d'apprentissage

In [43]:
from aif360.algorithms.preprocessing import Reweighing

model_reweighing = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

# Fit the reweighing model on the training dataset
model_reweighing.fit(dataset_orig_panel19_train)

# Transform the training dataset to get the reweighed version
dataset_reweighed = model_reweighing.transform(dataset_orig_panel19_train)

print("Reweighing transformation applied to the training dataset.")

Reweighing transformation applied to the training dataset.


#### 2.2.2. Question: Apprendre une regression logistique sur les données pondérées et calculer les métriques de fairness sur l'échantillon de validation

Comme vu en cours le Reweighting ne modifie que la pondération du dataset, les features et label restent inchangés.

In [ ]:
print("TODO")

### 2.3. Disparate Impact Remover
#### 2.3.1. Question : Trouver dans l'API quels objets/fonctions sont à utiliser pour faire une approache de disparate impact remover et les appliquer.


In [ ]:
print("TODO")

#### 2.3.2. Question: Apprendre une regression logistique sur les données transformées en retirant l'attribut sensible et calculer les métriques de fairness sur l'échantillon de validation

In [ ]:
print("TODO")

### 2.4. Question: Apprentissage de représentation latente fair

Apprendre le pre-processing et evaluer son impact avec les métriques

In [ ]:
print("TODO")

## 3 Post processing

### 3.1 Question: Use the post-processing Reject Option Classification

In [ ]:
print("TODO")

#### 3.1.1 Reuse the first Logistic Regression learn to find the best threshold that maximises its balanced accuracy on the validation dataset

In [ ]:
print("TODO")

#### 3.1.2 Use the RejectOptionClassification  on the validation dataset with the logistic regression predictions. To improve the fairness metrics

In [ ]:
print("TODO")

#### 3.1.3 Do the same while starting from the Logistic Regression learned on the Reweighted dataset

In [ ]:
print("TODO")

#### 3.2 Use the Calibrated Equalised Odds  on the validation dataset with the logistic regression predictions. To improve the fairness metrics

In [ ]:
print("TODO")